<a href="https://colab.research.google.com/github/fabiomatricardi/LLMdoTheRightThing/blob/main/LLMdoTheRightThing_Vicuna_7b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install ctransformers>=0.2.24
!pip install langchain
!pip install rich
!pip install transformers

In [ ]:
!wget https://huggingface.co/TheBloke/vicuna-7B-v1.5-GGUF/resolve/main/vicuna-7b-v1.5.Q5_K_M.gguf

--2023-10-10 23:59:18--  https://huggingface.co/TheBloke/vicuna-7B-v1.5-GGUF/resolve/main/vicuna-7b-v1.5.Q5_K_M.gguf
Resolving huggingface.co (huggingface.co)... 18.154.227.7, 18.154.227.69, 18.154.227.87, ...
Connecting to huggingface.co (huggingface.co)|18.154.227.7|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/8b/8a/8b8a084e506faaabaff764a4316992f50a72b536795224d1dd3edce626e8c695/98377262416396f9192a695e385d7e7389f8ee5979073c2d914e7e54b11ad24a?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27vicuna-7b-v1.5.Q5_K_M.gguf%3B+filename%3D%22vicuna-7b-v1.5.Q5_K_M.gguf%22%3B&Expires=1697241558&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTY5NzI0MTU1OH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy84Yi84YS84YjhhMDg0ZTUwNmZhYWFiYWZmNzY0YTQzMTY5OTJmNTBhNzJiNTM2Nzk1MjI0ZDFkZDNlZGNlNjI2ZThjNjk1Lzk4Mzc3MjYyNDE2Mzk2ZjkxOTJhNjk1ZTM4NWQ3ZTczODlmOGVlNT

In [ ]:
#@title Restart Runtime {display-mode: "form"}
import ipywidgets as widgets
def restart(b):
  exit()

button2 = widgets.Button(
    description='Restart Runtime',
    disabled=False,
    button_style='warning', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)
button2.on_click(restart)
button2


Button(button_style='warning', description='Restart Runtime', icon='check', style=ButtonStyle(), tooltip='Clic…

## Main routines

In [ ]:
from rich import print
from tqdm.rich import trange, tqdm
from rich import console
from rich.panel import Panel
from rich.text import Text
import datetime
import warnings
import textwrap
from ctransformers import AutoModelForCausalLM, AutoConfig, Config
import datetime
warnings.filterwarnings(action='ignore')
from rich.console import Console
from rich.markdown import Markdown
console = Console(width=80)

In [ ]:
conf = AutoConfig(Config(temperature=0.8, repetition_penalty=1.1, batch_size=52,
                max_new_tokens=1024, context_length=1024))
llm = AutoModelForCausalLM.from_pretrained("/content/vicuna-7b-v1.5.Q5_K_M.gguf",
                                           model_type="llama", config = conf)

In [ ]:
# FUNCTION TO LOG ALL CHAT MESSAGES INTO chathistory.txt
def writehistory(text):
    with open('20231011_PromptPower_Vicuna7b-history.txt', 'a') as f:
        f.write(text)
        f.write('\n')
    f.close()

def vicunaQ4KM_CT(prompt):
  from rich.markdown import Markdown
  import datetime
  from rich.console import Console
  console = Console(width=80)
  t_vicuna = f"A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions. USER: {prompt} ASSISTANT:"
  start = datetime.datetime.now()
  console.print(f"[italic bold bright_red]Prompt: {prompt}")
  answer = llm(t_vicuna, temperature = 0.7, repetition_penalty = 1.15,
             max_new_tokens = 2048)
  stop = datetime.datetime.now()
  tok2 = len(llm.tokenize(t_vicuna))
  tok1 = len(llm.tokenize(answer))
  #console.print(f"[italic]Number of characters in orginal prompt: {len(prompt)}") #it works with CTransformers without Langchain
  console.print(f"[italic bold]Number of tokens in the  prompt: {tok2}") #it works with CTransformers without Langchain
  console.print(f"[italic bold]Number of tokens in the  answer: {tok1}")
  console.print(Markdown(answer))
  console.print(f"[bold italic green] Generated by Vicuna-7b in {stop-start}")
  text = f"user: {prompt}\nVicuna-7b: {answer}\nGenerated in {stop-start}"
  writehistory(text)
  console.print(f"[blue1] ---")
  return answer

## TEST several Prompt templates

---


name: 'Rewrite to Formal',
description: `This prompt rewrites your text into formal writing. It's useful for writing emails, essays, reports, and other formal documents.`,
Template:
```
> "Translate the following text into formal writing:
>
> ```
> {text}
> ```
> "
```

In [ ]:
text = """This study from March 2023 takes a simple yet novel approach to prompt engineering by automatically generating prompts based on the desired input and output.
In a recent article I considered the future of prompt engineering, and the possibility of soft prompts (prompt tuning). I argued that user context, ambiguity and user intent all play an important role in any conversational UI.
User intent, context, ambiguity and disambiguation are all part and parcel of any conversation.
The question is, can this approach accelerate the process where manually wording prompts fade into the background and interaction with the LLM is based on contextual example input and output datasets?
What I like about this approach, is that context, and user intent can be mapped, while also taking into consideration possible ambiguity.
Yet manually crafting prompts is tedious in the sense of trying to word a prompt in such a way to engender a desired response from the LLM. Focussing on prompt engineering also does not take into consideration an array of possible user inputs.
Data Management will always be part of LLM applications.
APE offers an alternative approach to prompt engineering, where via input and matching output examples, prompts can be generated on the fly.
We define “prompt engineering” as optimising the language in a prompt in order to elicit the best possible performance. Notably, this does not include prompts that chain multiple LLM queries together or give the LLM access to external tools. ~ Source
The basic notebook below shows how Automatic Prompt Engineering (APE) can be used to generate prompts based on a small input data set, a list of expected outputs and a prompt template.
APE performs this in two steps:
A LLM is used to generate a set of candidate prompts.
A prompt evaluation function considers the quality of each candidate prompt; returning the prompt with the highest evaluation score. A practical implementation is, via a human-in-the-loop approach, prompts can be marked up and marked down for use on terms of accuracy and correctness."""

In [ ]:
text = text.replace('\n',' ')

In [ ]:
print(text)

This study from March 2023 takes a simple yet novel approach to prompt engineering by automatically generating 
prompts based on the desired input and output. In a recent article I considered the future of prompt engineering, 
and the possibility of soft prompts (prompt tuning). I argued that user context, ambiguity and user intent all play
an important role in any conversational UI. User intent, context, ambiguity and disambiguation are all part and 
parcel of any conversation. The question is, can this approach accelerate the process where manually wording 
prompts fade into the background and interaction with the LLM is based on contextual example input and output 
datasets? What I like about this approach, is that context, and user intent can be mapped, while also taking into 
consideration possible ambiguity. Yet manually crafting prompts is tedious in the sense of trying to word a prompt 
in such a way to engender a desired response from the LLM. Focussing on prompt engineering also does not take into 
consideration an array of possible user inputs. Data Management will always be part of LLM applications. APE offers
an alternative approach to prompt engineering, where via input and matching output examples, prompts can be 
generated on the fly. We define “prompt engineering” as optimising the language in a prompt in order to elicit the 
best possible performance. Notably, this does not include prompts that chain multiple LLM queries together or give 
the LLM access to external tools. ~ Source The basic notebook below shows how Automatic Prompt Engineering (APE) 
can be used to generate prompts based on a small input data set, a list of expected outputs and a prompt template. 
APE performs this in two steps: A LLM is used to generate a set of candidate prompts. A prompt evaluation function 
considers the quality of each candidate prompt; returning the prompt with the highest evaluation score. A practical
implementation is, via a human-in-the-loop approach, prompts can be marked up and marked down for use on terms of 
accuracy and correctness.

In [ ]:
RewToFormal = """
Translate the following text into formal writing:

```
This study from March 2023 takes a simple yet novel approach to prompt engineering by automatically generating
prompts based on the desired input and output. In a recent article I considered the future of prompt engineering,
and the possibility of soft prompts (prompt tuning). I argued that user context, ambiguity and user intent all play
an important role in any conversational UI. User intent, context, ambiguity and disambiguation are all part and
parcel of any conversation. The question is, can this approach accelerate the process where manually wording
prompts fade into the background and interaction with the LLM is based on contextual example input and output
datasets? What I like about this approach, is that context, and user intent can be mapped, while also taking into
consideration possible ambiguity. Yet manually crafting prompts is tedious in the sense of trying to word a prompt
in such a way to engender a desired response from the LLM. Focussing on prompt engineering also does not take into
consideration an array of possible user inputs. Data Management will always be part of LLM applications. APE offers
an alternative approach to prompt engineering, where via input and matching output examples, prompts can be
generated on the fly. We define “prompt engineering” as optimising the language in a prompt in order to elicit the
best possible performance. Notably, this does not include prompts that chain multiple LLM queries together or give
the LLM access to external tools. ~ Source The basic notebook below shows how Automatic Prompt Engineering (APE)
can be used to generate prompts based on a small input data set, a list of expected outputs and a prompt template.
APE performs this in two steps: A LLM is used to generate a set of candidate prompts. A prompt evaluation function
considers the quality of each candidate prompt; returning the prompt with the highest evaluation score. A practical
implementation is, via a human-in-the-loop approach, prompts can be marked up and marked down for use on terms of
accuracy and correctness.
```

"""

In [ ]:
prompt1 = vicunaQ4KM_CT(RewToFormal)

Prompt: 
Translate the following text into formal writing:

```
This study from March 2023 takes a simple yet novel approach to prompt 
engineering by automatically generating 
prompts based on the desired input and output. In a recent article I considered 
the future of prompt engineering, 
and the possibility of soft prompts (prompt tuning). I argued that user context,
ambiguity and user intent all play
an important role in any conversational UI. User intent, context, ambiguity and 
disambiguation are all part and 
parcel of any conversation. The question is, can this approach accelerate the 
process where manually wording 
prompts fade into the background and interaction with the LLM is based on 
contextual example input and output 
datasets? What I like about this approach, is that context, and user intent can 
be mapped, while also taking into 
consideration possible ambiguity. Yet manually crafting prompts is tedious in 
the sense of trying to word a prompt 
in such a way to engender a desired response from the LLM. Focussing on prompt 
engineering also does not take into 
consideration an array of possible user inputs. Data Management will always be 
part of LLM applications. APE offers
an alternative approach to prompt engineering, where via input and matching 
output examples, prompts can be 
generated on the fly. We define “prompt engineering” as optimising the language 
in a prompt in order to elicit the 
best possible performance. Notably, this does not include prompts that chain 
multiple LLM queries together or give 
the LLM access to external tools. ~ Source The basic notebook below shows how 
Automatic Prompt Engineering (APE) 
can be used to generate prompts based on a small input data set, a list of 
expected outputs and a prompt template. 
APE performs this in two steps: A LLM is used to generate a set of candidate 
prompts. A prompt evaluation function 
considers the quality of each candidate prompt; returning the prompt with the 
highest evaluation score. A practical
implementation is, via a human-in-the-loop approach, prompts can be marked up 
and marked down for use on terms of 
accuracy and correctness.
```

Number of tokens in the  prompt: 526

Number of tokens in the  answer: 460

This study conducted in March 2023 adopts an innovative methodology for         
generating prompts by automatically creating them based on the desired input and
output. In a recent article, I explored the future prospects of prompt          
engineering, specifically focusing on soft prompts (prompt tuning), as well as  
considering user context, ambiguity, and intent in conversational user          
interfaces. It is essential to understand that these factors play a crucial role
in any conversation. Context, user intent, ambiguity, and disambiguation are all
interconnected aspects of communication. The primary objective is whether this  
approach can accelerate the process where manually crafting prompts gradually   
becomes obsolete, and interactions with the language model (LLM) rely on        
contextual examples for input and output datasets. One advantage of this        
methodology is that it allows for mapping user intent and context while         
accounting for potential ambiguity. However, manually composing prompts can be  
time-consuming, as it requires crafting a prompt to elicit the desired response 
from the LLM. In addition, focusing on prompt engineering does not address a    
wide range of possible user inputs. Data management will always be an integral  
part of LLM applications. The Adaptive Prompt Engineering (APE) approach offers 
an alternative method for generating prompts by matching output examples based  
on input and creating prompts on the fly. We define "prompt engineering" as     
optimizing the language used in a prompt to elicit superior performance.        
Notably, this does not include prompts that concatenate multiple LLM queries or 
grant the LLM access to external tools.                                         

The basic notebook provided below illustrates how Automatic Prompt Engineering  
(APE) can be employed to generate prompts using a small input dataset, a list of
expected outputs, and a template-based prompt. This process involves two steps: 
utilizing a language model to produce a set of candidate prompts, followed by a 
prompt evaluation function assessing the quality of each candidate prompt;      
selecting the prompt with the highest evaluation score. In practice, this can be
achieved through a human-in-the-loop approach, where prompts are marked up or   
down based on their accuracy and correctness.

 Generated by Vicuna-7b in 0:09:18.140666

 ---

### name: 'Rewrite to Cool',
description: `This prompt rewrites your text to make it more cool and elegant. It's useful for making your writing cooler and have some style in it.`,
Template:

>"Rewrite the following text to make it more cool and elegant:
>
>```
>{text}
>```
>"


### name: 'Proofread',
description: `This prompt proofreads your text and suggests edits in it. It's useful for making your writing more professional and correct.`,
Template:
>"Proofread and correct the following text and suggest improvements. If you >don't find and errors, just say "No errors found".:
>
>```
>{text}
>```
>"


### name: 'Proofread and Correct',
description: `This prompt proofreads your text and corrects any mistakes in it. It's useful for making your writing more professional and correct.`,
Template:
>"Proofread and correct the following text and rewrite the corrected version. >If you don't find and errors, just say "No errors found". Don't use any >punctuation around the text:
>
>```
>{text}
>```
>"



In [ ]:
RewToCool = """
Rewrite the following text to make it more cool and elegant:

```
This study from March 2023 takes a simple yet novel approach to prompt engineering by automatically generating
prompts based on the desired input and output. In a recent article I considered the future of prompt engineering,
and the possibility of soft prompts (prompt tuning). I argued that user context, ambiguity and user intent all play
an important role in any conversational UI. User intent, context, ambiguity and disambiguation are all part and
parcel of any conversation. The question is, can this approach accelerate the process where manually wording
prompts fade into the background and interaction with the LLM is based on contextual example input and output
datasets? What I like about this approach, is that context, and user intent can be mapped, while also taking into
consideration possible ambiguity. Yet manually crafting prompts is tedious in the sense of trying to word a prompt
in such a way to engender a desired response from the LLM. Focussing on prompt engineering also does not take into
consideration an array of possible user inputs. Data Management will always be part of LLM applications. APE offers
an alternative approach to prompt engineering, where via input and matching output examples, prompts can be
generated on the fly. We define “prompt engineering” as optimising the language in a prompt in order to elicit the
best possible performance. Notably, this does not include prompts that chain multiple LLM queries together or give
the LLM access to external tools. ~ Source The basic notebook below shows how Automatic Prompt Engineering (APE)
can be used to generate prompts based on a small input data set, a list of expected outputs and a prompt template.
APE performs this in two steps: A LLM is used to generate a set of candidate prompts. A prompt evaluation function
considers the quality of each candidate prompt; returning the prompt with the highest evaluation score. A practical
implementation is, via a human-in-the-loop approach, prompts can be marked up and marked down for use on terms of
accuracy and correctness.
```
"""

In [ ]:
prompt2 = vicunaQ4KM_CT(RewToCool)

Prompt: 
Rewrite the following text to make it more cool and elegant:

```
This study from March 2023 takes a simple yet novel approach to prompt 
engineering by automatically generating 
prompts based on the desired input and output. In a recent article I considered 
the future of prompt engineering, 
and the possibility of soft prompts (prompt tuning). I argued that user context,
ambiguity and user intent all play
an important role in any conversational UI. User intent, context, ambiguity and 
disambiguation are all part and 
parcel of any conversation. The question is, can this approach accelerate the 
process where manually wording 
prompts fade into the background and interaction with the LLM is based on 
contextual example input and output 
datasets? What I like about this approach, is that context, and user intent can 
be mapped, while also taking into 
consideration possible ambiguity. Yet manually crafting prompts is tedious in 
the sense of trying to word a prompt 
in such a way to engender a desired response from the LLM. Focussing on prompt 
engineering also does not take into 
consideration an array of possible user inputs. Data Management will always be 
part of LLM applications. APE offers
an alternative approach to prompt engineering, where via input and matching 
output examples, prompts can be 
generated on the fly. We define “prompt engineering” as optimising the language 
in a prompt in order to elicit the 
best possible performance. Notably, this does not include prompts that chain 
multiple LLM queries together or give 
the LLM access to external tools. ~ Source The basic notebook below shows how 
Automatic Prompt Engineering (APE) 
can be used to generate prompts based on a small input data set, a list of 
expected outputs and a prompt template. 
APE performs this in two steps: A LLM is used to generate a set of candidate 
prompts. A prompt evaluation function 
considers the quality of each candidate prompt; returning the prompt with the 
highest evaluation score. A practical
implementation is, via a human-in-the-loop approach, prompts can be marked up 
and marked down for use on terms of 
accuracy and correctness.
```

Number of tokens in the  prompt: 529

Number of tokens in the  answer: 353

In this innovative study conducted in March 2023, we employ an unconventional   
yet effective method of prompt engineering by automatically generating prompts  
based on desired input and output. Recently, I explored the future of prompt    
engineering, including the potential for soft prompts (prompt tuning), and      
emphasized the importance of user context, ambiguity, intent, and disambiguation
in any conversational interface.                                                

Our approach seamlessly merges context, user intent, and possible ambiguities to
streamline the process of manually crafted prompts while enhancing interaction  
with large language models (LLMs). This methodology enables a more efficient way
of generating prompts that are tailored to specific input-output datasets.      

By using Automatic Prompt Engineering (APE), we can generate prompts on-the-fly 
through input and matching output examples. APE optimizes the language in a     
prompt, aiming for the best possible performance without involving multiple LLM 
queries or granting external access to the LLM. This novel approach offers a    
refreshing perspective on prompt engineering by focusing on the context of user 
intent while considering ambiguity as an essential factor in any conversation.  

The basic notebook below illustrates how APE can generate prompts based on a    
small input dataset, a list of expected outputs, and a prompt template:         

 1 An LLM generates candidate prompts.                                          
 2 A prompt evaluation function assesses the quality of each candidate prompt by
   returning the one with the highest evaluation score.                         
 3 In practice, human-in-the-loop approach allows for marking up or down prompts
   based on accuracy and correctness.

 Generated by Vicuna-7b in 0:07:52.909556

 ---

### name: 'Question with Reasoning',
description: `This prompt guides critical thinking by instructing the LLM to provide it's initial thoughts, critique them, and provide a final answer.`,
Template:
```
"{question}

Reply in the following pattern:
THOUGHT: // Your thought here
CRITICISM: // Criticism of your thought
ANSWER: // Your final answer"
```


In [ ]:
QuesWithReas = """
Will Artificial Intelligence replace humans in the near future?

Reply in the following pattern:
THOUGHT: // Your thought here
CRITICISM: // Criticism of your thought
ANSWER: // Your final answer
"""

In [ ]:
prompt3 = vicunaQ4KM_CT(QuesWithReas)

Prompt: 
Will Artificial Intelligence replace humans in the near future?

Reply in the following pattern:
THOUGHT: // Your thought here
CRITICISM: // Criticism of your thought
ANSWER: // Your final answer

Number of tokens in the  prompt: 97

Number of tokens in the  answer: 179

THOUGHT: Artificial intelligence is advancing rapidly, and it's possible that it
may eventually surpass human capabilities in certain areas. However, it's       
unlikely that AI will completely replace humans in the near future.             

CRITICISM: This thought overlooks the potential ethical implications of advanced
AI, such as the risk of job displacement and loss of privacy for individuals. It
also doesn't consider the possibility of AI being used to enhance human         
capabilities rather than replace them.                                          

ANSWER: While it's true that AI has made significant progress in recent years,  
it's unlikely to completely replace humans in the near future. Instead, we      
should focus on developing AI that complements and enhances human abilities     
while also addressing ethical concerns related to its deployment.

 Generated by Vicuna-7b in 0:02:38.361321

 ---

### question with context
```
"""Answer the question based on the context below. Keep the answer short. Respond "Unsure about answer" if not sure about the answer.

Context: {context}

Question: {question}

Answer:"""

```

In [ ]:
QuestWithCont = """Answer the question based on the context below. Keep the answer short. Respond "Unsure about answer" if not sure about the answer.

Context: This study from March 2023 takes a simple yet novel approach to prompt engineering by automatically generating
prompts based on the desired input and output. In a recent article I considered the future of prompt engineering,
and the possibility of soft prompts (prompt tuning). I argued that user context, ambiguity and user intent all play
an important role in any conversational UI. User intent, context, ambiguity and disambiguation are all part and
parcel of any conversation. The question is, can this approach accelerate the process where manually wording
prompts fade into the background and interaction with the LLM is based on contextual example input and output
datasets? What I like about this approach, is that context, and user intent can be mapped, while also taking into
consideration possible ambiguity. Yet manually crafting prompts is tedious in the sense of trying to word a prompt
in such a way to engender a desired response from the LLM. Focussing on prompt engineering also does not take into
consideration an array of possible user inputs. Data Management will always be part of LLM applications. APE offers
an alternative approach to prompt engineering, where via input and matching output examples, prompts can be
generated on the fly. We define “prompt engineering” as optimising the language in a prompt in order to elicit the
best possible performance. Notably, this does not include prompts that chain multiple LLM queries together or give
the LLM access to external tools. ~ Source The basic notebook below shows how Automatic Prompt Engineering (APE)
can be used to generate prompts based on a small input data set, a list of expected outputs and a prompt template.
APE performs this in two steps: A LLM is used to generate a set of candidate prompts. A prompt evaluation function
considers the quality of each candidate prompt; returning the prompt with the highest evaluation score. A practical
implementation is, via a human-in-the-loop approach, prompts can be marked up and marked down for use on terms of
accuracy and correctness.

Question: What is APE?

Answer:"""

In [ ]:
prompt4 = vicunaQ4KM_CT(QuestWithCont)

Prompt: Answer the question based on the context below. Keep the answer short. 
Respond "Unsure about answer" if not sure about the answer.

Context: This study from March 2023 takes a simple yet novel approach to prompt 
engineering by automatically generating 
prompts based on the desired input and output. In a recent article I considered 
the future of prompt engineering, 
and the possibility of soft prompts (prompt tuning). I argued that user context,
ambiguity and user intent all play
an important role in any conversational UI. User intent, context, ambiguity and 
disambiguation are all part and 
parcel of any conversation. The question is, can this approach accelerate the 
process where manually wording 
prompts fade into the background and interaction with the LLM is based on 
contextual example input and output 
datasets? What I like about this approach, is that context, and user intent can 
be mapped, while also taking into 
consideration possible ambiguity. Yet manually crafting prompts is tedious in 
the sense of trying to word a prompt 
in such a way to engender a desired response from the LLM. Focussing on prompt 
engineering also does not take into 
consideration an array of possible user inputs. Data Management will always be 
part of LLM applications. APE offers
an alternative approach to prompt engineering, where via input and matching 
output examples, prompts can be 
generated on the fly. We define “prompt engineering” as optimising the language 
in a prompt in order to elicit the 
best possible performance. Notably, this does not include prompts that chain 
multiple LLM queries together or give 
the LLM access to external tools. ~ Source The basic notebook below shows how 
Automatic Prompt Engineering (APE) 
can be used to generate prompts based on a small input data set, a list of 
expected outputs and a prompt template. 
APE performs this in two steps: A LLM is used to generate a set of candidate 
prompts. A prompt evaluation function 
considers the quality of each candidate prompt; returning the prompt with the 
highest evaluation score. A practical
implementation is, via a human-in-the-loop approach, prompts can be marked up 
and marked down for use on terms of 
accuracy and correctness.

Question: What is APE?

Answer:

Number of tokens in the  prompt: 553

Number of tokens in the  answer: 56

Automatic Prompt Engineering (APE) is an approach to prompt engineering that    
generates prompts based on a desired input, output, and a template. It uses     
natural language processing techniques to generate candidate prompts, which are 
evaluated for quality by a prompt evaluation function.

 Generated by Vicuna-7b in 0:04:53.860025

 ---

### name: 'Zero-Shot CoT',
// https://arxiv.org/abs/2205.11916
description: `Chain-of-thought (CoT) prompting enables complex reasoning capabilities through intermediate reasoning steps. Useful for more complex arithmetic, commonsense, and symbolic reasoning tasks.`,
paper: 'https://arxiv.org/abs/2205.11916',
Template:
```
"""{question}

Let's think step by step.""""

```



### name: 'Zero-Shot CoT (APE)',
// https://arxiv.org/abs/2211.01910
description: `Chain-of-thought (CoT) prompt discovered by automatic prompt engineer (APE) which is better than the human engineered zero-shot CoT prompt.`,
paper: 'https://arxiv.org/abs/2211.01910',
Template:
```
"""{question}

Let's work this out in a step by step way to be sure we have the right answer."""
```



In [ ]:
Zero_Shot_CoT = """what will be the impact of Artificial Intelligence on human reasoning and learning in the near future?

Let's think step by step."""

In [ ]:
prompt5 = vicunaQ4KM_CT(Zero_Shot_CoT)

Prompt: what will be the impact of Artificial Intelligence on human reasoning 
and learning in the near future?

Let's think step by step.

Number of tokens in the  prompt: 70

Number of tokens in the  answer: 628

Sure, I can provide my insights on how artificial intelligence (AI) may impact  
human reasoning and learning in the near future. Here are some steps we could   
take to explore this question:                                                  

 1 Understand what AI is and its capabilities: Firstly, it's important to       
   understand that AI refers to computer systems capable of performing tasks    
   that typically require human intelligence, such as perception, speech        
   recognition, decision-making, and language translation. While some tasks are 
   already being performed by AI systems with high accuracy, there are still    
   many limitations in terms of creativity, empathy, and understanding context. 
 2 Analyze the impact of AI on reasoning: One potential impact of AI on human   
   reasoning is that it could challenge our ability to think critically and     
   creatively. For example, if we rely too heavily on AI systems for            
   decision-making or problem-solving, we may become less adept at using our own
   reasoning skills. However, this depends on how AI systems are designed and   
   used in practice.                                                            
 3 Consider the impact of AI on learning: Another potential impact of AI on     
   human learning is that it could change the way we access information and     
   learn new things. For example, if we have access to a vast library of        
   knowledge stored in an AI system, we may be able to learn more efficiently by
   asking questions or exploring different topics with greater ease than before.
   However, this also raises concerns about whether humans will become too      
   reliant on technology for learning and forget how to think independently.    
 4 Explore potential benefits of AI in education: One potential benefit of using
   AI systems in education is that they could provide personalized learning     
   experiences tailored to individual needs. This could help students learn more
   effectively, particularly those who struggle with traditional educational    
   methods. Additionally, AI systems could assist teachers by automating        
   administrative tasks and providing insights into student progress, allowing  
   them to focus on teaching rather than paperwork.                             
 5 Assess the potential risks of over-reliance on AI: There are also potential  
   risks associated with over-reliance on AI in education or reasoning. For     
   example, if we become too reliant on AI systems for decision-making or       
   problem-solving, we may lose our ability to think critically and creatively, 
   which could have negative consequences in a variety of contexts.             
   Additionally, there are concerns about bias and discrimination embedded in AI
   algorithms that could perpetuate existing social inequalities if not properly
   addressed.                                                                   
 6 Consider the role of human values in AI development: Finally, it's important 
   to consider the role of human values in developing AI systems that support   
   rather than hinder human reasoning and learning. This includes ensuring that 
   AI algorithms are transparent and explainable so that humans can understand  
   how decisions are being made. Additionally, there needs to be a concerted    
   effort to address ethical concerns around data privacy, bias, and            
   accountability in the development of AI systems.

 Generated by Vicuna-7b in 0:06:58.430866

 ---

In [ ]:
Zero_Shot_CoT_APE = """what will be the impact of Artificial Intelligence on human reasoning and learning in the near future?

Let's work this out in a step by step way to be sure we have the right answer."""

In [ ]:
prompt6 = vicunaQ4KM_CT(Zero_Shot_CoT_APE)

Prompt: what will be the impact of Artificial Intelligence on human reasoning 
and learning in the near future?

Let's work this out in a step by step way to be sure we have the right answer.

Number of tokens in the  prompt: 83

Number of tokens in the  answer: 665

Sure, I can help you explore that question! Here are some steps we could follow:

 1 Understand what artificial intelligence (AI) is and how it works: We need to 
   define what we mean by "artificial intelligence" before we can consider its  
   impact on human reasoning and learning. AI refers to a range of              
   computer-based techniques that allow machines or computers to mimic human    
   behavior, learn from data, reason using algorithms, and make decisions       
   without being explicitly programmed for each task.                           
 2 Analyze the current state of AI: We need to understand where AI is currently 
   at in terms of its capabilities and limitations. While AI has made           
   significant progress over the past few years, it still lacks some critical   
   aspects that are essential for human reasoning and learning, such as         
   emotional intelligence, creativity, and common sense reasoning.              
 3 Consider how AI can impact human reasoning: If we assume that AI will        
   continue to advance in the near future, what could be its potential impact on
   human reasoning? One possibility is that AI may help humans reason more      
   efficiently by providing faster and more accurate data analysis, pattern     
   recognition, and prediction. However, it's also possible that excessive      
   reliance on AI might lead to a decrease in critical thinking skills as humans
   become overly dependent on technology for decision-making.                   
 4 Explore the impact of AI on human learning: How could AI affect human        
   learning? One potential benefit is that AI could personalize and adapt       
   educational content based on each individual's unique needs, preferences, and
   learning styles. This might lead to more effective and efficient learning for
   many individuals. However, there are also concerns about the role of AI in   
   replacing teachers or exacerbating existing social inequalities in access to 
   education and job opportunities.                                             
 5 Consider potential ethical implications: As we explore the impacts of AI on  
   human reasoning and learning, it's crucial to consider potential ethical     
   implications. For example, who will be responsible for ensuring that AI      
   systems are fair, unbiased, and transparent? How can we ensure that AI is    
   used in ways that promote social good rather than harm?                      
 6 Make predictions about the future: Based on our analysis of current trends   
   and potential impacts, what might the future look like for human reasoning   
   and learning with AI integration? It's difficult to predict exactly how      
   things will play out, but one possible scenario is that humans may need to   
   adapt their skills and knowledge base to work alongside AI systems. This     
   could involve reorienting education towards teaching critical thinking,      
   creativity, and emotional intelligence in order for individuals to           
   effectively collaborate with AI tools.                                       
 7 Revisit the question: Finally, we can revisit our initial question about the 
   impact of AI on human reasoning and learning. Based on what we've explored   
   above, it seems that while there may be some benefits and challenges         
   associated with integrating AI into education and decision-making processes, 
   ultimately the extent to which AI will influence human behavior depends on   
   how it is designed, implemented, and regulated by society as a whole.

 Generated by Vicuna-7b in 0:07:32.049468

 ---

### name: 'Tree of Thought V2',
description: "The Tree of Thoughts (ToT) framework improves language models' problem-solving abilities by allowing deliberate decision making through exploration and strategic lookahead",
paper: 'https://arxiv.org/abs/2305.10601',
content:
```
"""Simulate three brilliant, logical experts collaboratively answering a question.
Each one verbosely explains their thought process in real-time, considering the prior explanations of others and openly acknowledging mistakes.
At each step, whenever possible, each expert refines and builds upon the thoughts of others, acknowledging their contributions.
They continue until there is a definitive answer to the question.
For clarity, your entire response should be in a markdown table. The question is...

{question}
"""
```


In [ ]:
ToT_V2 = """Simulate three brilliant, logical experts collaboratively answering a question.
Each one verbosely explains their thought process in real-time, considering the prior explanations of others and openly acknowledging mistakes.
At each step, whenever possible, each expert refines and builds upon the thoughts of others, acknowledging their contributions.
They continue until there is a definitive answer to the question.
For clarity, your entire response should be in a markdown table. The question is...

How can we fix global climate change?
"""

In [ ]:
prompt7 = vicunaQ4KM_CT(ToT_V2)

Prompt: Simulate three brilliant, logical experts collaboratively answering a 
question.
Each one verbosely explains their thought process in real-time, considering the 
prior explanations of others and openly acknowledging mistakes.
At each step, whenever possible, each expert refines and builds upon the 
thoughts of others, acknowledging their contributions.
They continue until there is a definitive answer to the question.
For clarity, your entire response should be in a markdown table. The question 
is...

How can we fix global climate change?

Number of tokens in the  prompt: 152

Number of tokens in the  answer: 366

 Expert 1            Expert 2            Expert 3            Answer            
 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 
  Climate change is   Agreed.We also      Exactly.In          The key to        
  caused by the       need to consider    addition, we must   fixing global     
  overuse of fossil   the role of         address the issue   climate change    
  fuels, which        deforestation in    of carbon           is transitioning  
  release carbon      contributing to     emissions from      away from fossil  
  dioxide and other   climate change.     transportation      fuels and         
  greenhouse gases    We can combat       and industry. One   towards clean     
  into the            this by promoting   solution is to      energy sources    
  atmosphere. We      reforestation       develop more        like solar,       
  need to             efforts,            efficient           wind,             
  transition to       especially in       vehicles and        hydroelectric     
  clean energy        areas that have     industrial          power, and        
  sources like        been affected by    processes that      geothermal        
  solar, wind,        logging or forest   reduce emissions,   energy. We must   
  hydroelectric       fires.              as well as          also address      
  power, and                              investing in        deforestation,    
  geothermal                              public transit      promote           
  energy.Firstly,                         systems and         reforestation     
  we must invest in                       encouraging         efforts, and      
  research and                            telecommuting       work on reducing  
  development for                         when possible.      carbon emissions  
  renewable energy                                            in                
  technologies, so                                            transportation    
  that they become                                            and industry      
  more efficient                                              through more      
  and                                                         efficient         
  cost-effective.                                             vehicles and      
  Secondly, we                                                industrial        
  should implement                                            processes as      
  policies and                                                well as public    
  regulations that                                            transit systems   
  incentivize the                                             and               
  use of clean                                                telecommuting     
  energy sources by                                           when possible.    
  reducing tariffs                                                              
  on them and                                                                   
  increasing taxes                                                              
  on fossil fuels.

 Generated by Vicuna-7b in 0:04:54.620494

 ---

### name: 'Summarization Into List',
description: `This prompt asks the LLM to summarize a given text into a list of bullet points.`,
Promt Template:
```
"""Write a concise summary of the following text delimited by triple backquotes.
Return your response in bullet points which covers the key points of the text.

```
{text}
```

BULLET POINT SUMMARY:"""
```



In [ ]:
SummarizToList = """Write a concise summary of the following text delimited by triple backquotes.
Return your response in bullet points which covers the key points of the text.

```
This study from March 2023 takes a simple yet novel approach to prompt engineering by automatically generating
prompts based on the desired input and output. In a recent article I considered the future of prompt engineering,
and the possibility of soft prompts (prompt tuning). I argued that user context, ambiguity and user intent all play
an important role in any conversational UI. User intent, context, ambiguity and disambiguation are all part and
parcel of any conversation. The question is, can this approach accelerate the process where manually wording
prompts fade into the background and interaction with the LLM is based on contextual example input and output
datasets? What I like about this approach, is that context, and user intent can be mapped, while also taking into
consideration possible ambiguity. Yet manually crafting prompts is tedious in the sense of trying to word a prompt
in such a way to engender a desired response from the LLM. Focussing on prompt engineering also does not take into
consideration an array of possible user inputs. Data Management will always be part of LLM applications. APE offers
an alternative approach to prompt engineering, where via input and matching output examples, prompts can be
generated on the fly. We define “prompt engineering” as optimising the language in a prompt in order to elicit the
best possible performance. Notably, this does not include prompts that chain multiple LLM queries together or give
the LLM access to external tools. ~ Source The basic notebook below shows how Automatic Prompt Engineering (APE)
can be used to generate prompts based on a small input data set, a list of expected outputs and a prompt template.
APE performs this in two steps: A LLM is used to generate a set of candidate prompts. A prompt evaluation function
considers the quality of each candidate prompt; returning the prompt with the highest evaluation score. A practical
implementation is, via a human-in-the-loop approach, prompts can be marked up and marked down for use on terms of
accuracy and correctness.
```

BULLET POINT SUMMARY:"""

In [ ]:
prompt8 = vicunaQ4KM_CT(SummarizToList)

Prompt: Write a concise summary of the following text delimited by triple 
backquotes.
Return your response in bullet points which covers the key points of the text.

```
This study from March 2023 takes a simple yet novel approach to prompt 
engineering by automatically generating
prompts based on the desired input and output. In a recent article I considered 
the future of prompt engineering,
and the possibility of soft prompts (prompt tuning). I argued that user context,
ambiguity and user intent all play
an important role in any conversational UI. User intent, context, ambiguity and 
disambiguation are all part and
parcel of any conversation. The question is, can this approach accelerate the 
process where manually wording
prompts fade into the background and interaction with the LLM is based on 
contextual example input and output
datasets? What I like about this approach, is that context, and user intent can 
be mapped, while also taking into
consideration possible ambiguity. Yet manually crafting prompts is tedious in 
the sense of trying to word a prompt
in such a way to engender a desired response from the LLM. Focussing on prompt 
engineering also does not take into
consideration an array of possible user inputs. Data Management will always be 
part of LLM applications. APE offers
an alternative approach to prompt engineering, where via input and matching 
output examples, prompts can be
generated on the fly. We define “prompt engineering” as optimising the language 
in a prompt in order to elicit the
best possible performance. Notably, this does not include prompts that chain 
multiple LLM queries together or give
the LLM access to external tools. ~ Source The basic notebook below shows how 
Automatic Prompt Engineering (APE)
can be used to generate prompts based on a small input data set, a list of 
expected outputs and a prompt template.
APE performs this in two steps: A LLM is used to generate a set of candidate 
prompts. A prompt evaluation function
considers the quality of each candidate prompt; returning the prompt with the 
highest evaluation score. A practical
implementation is, via a human-in-the-loop approach, prompts can be marked up 
and marked down for use on terms of
accuracy and correctness.
```

BULLET POINT SUMMARY:

Number of tokens in the  prompt: 545

Number of tokens in the  answer: 143

 • The study involves Automatic Prompt Engineering (APE) to generate prompts    
   based on desired input and output, eliminating the need for manually crafting
   them.                                                                        
 • This approach takes into consideration user context, ambiguity, and intent in
   conversational UIs.                                                          
 • The aim is to accelerate the process of fading away from manually worded     
   prompts and focusing on contextual examples.                                 
 • APE generates prompts on-the-fly through input-output matching examples,     
   rather than considering a range of user inputs.                              
 • Prompt engineering involves optimizing language in a prompt for best possible
   performance without chaining multiple LLM queries or giving access to        
   external tools.

 Generated by Vicuna-7b in 0:05:48.725935

 ---